In [1]:
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_community.llms import HuggingFaceHub
from langchain_groq import ChatGroq
import pickle

In [1]:
import pandas as pd

df1 = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/ragas_dataset_rag_exp_llm_chain_filter(4)(1).csv')


In [3]:
df = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/final_selection.csv')


In [2]:
df1

,question,answer,contexts,ground_truth,original_contexts
0,What operations can be performed related to da...,"Based on the provided context, the following o...",['11 Delete an existing configuration. 12 Modi...,"Users can restore backups, create backups, con...",The database management section of the service...
1,What word is written below the symbol in the t...,The word written below the symbol in the third...,['The third icon on the right has a bright gre...,"The word ""AUTOMATIC"" is written below the symb...",The text describes various icons and their fun...
2,What option allows data to be deleted on unuse...,The option that allows data to be deleted on u...,['1. Tabs for different settings categories c...,"An option that, if selected, allows data to be...",The text describes the main elements and funct...
3,Which item would you consult to learn how to u...,"You would consult item 3, ""Manuals for machine...",['Show the menu. 2. Selection of the Tulus gui...,Manuals for machines and devices,The text content lists items such as showing t...
4,What information is included in the header bar...,The header bar of the interface includes a blu...,['A blue horizontal bar stretches across the i...,The header bar includes information about the ...,The text describes a user interface for managi...
...,...,...,...,...,...
121,What does high humidity level vs temp in Preci...,"Based on the context, high humidity level in t...","['Alarm ID is: 2659, Source type is: PLC alarm...",The alarm 'Precitec Collimator - Humidity Max ...,"['Alarm ID is: 2659, Source type is: PLC alarm..."
122,"What does Alarm ID 3015 (user warning, axis ov...","Unfortunately, I don't have any context to pro...",[],Alarm ID 3015 indicates an 'Auxiliary Axis in ...,"['Alarm ID is: 3015, Source type is: Servo ala..."
123,What can you tell me about alarms 20/16 & 23/3...,"Unfortunately, there is no context provided to...",[],Alarm 20/16 and 23/363 are CNC alarms with an ...,"['Alarm ID is: 20/16, Source type is: CNC alar..."
124,What common issue triggers 'Drive axis X1 X2 n...,Although the exact alarms 'Drive axis X1 X2 no...,"['Alarm ID is: 3064, Source type is: Servo ala...",A common issue triggering both 'Drive axis X1 ...,"['Alarm ID is: 15, Source type is: PLC alarm/m..."


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")
groq_api_key = os.environ["GROQ_API_KEY"]

# Naive RAG

In [3]:
hf=HuggingFaceHub(
    repo_id="meta-llama/Meta-Llama-3-70B",
    model_kwargs={"temperature":0.1}

)

/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama3-70b-8192")

In [111]:
query = "What are the advantages of a 3D printer?"

prompt = f"Refine the following query for better retrieval. Only return the refined query, without any additional text or explanation:\n\n{query}"

# Get the refined query from the LLM
response = llm.invoke(prompt)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [112]:
response.content

'What are the benefits and advantages of using 3D printing technology?'

In [62]:
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/data/chunks.pkl"
with open(pickle_file, "rb") as file:
    docs_manual = pickle.load(file)

In [63]:
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/data/alarm_info_list.pkl"
with open(pickle_file, "rb") as file:
    docs_alarm = pickle.load(file)

In [66]:
docs_list = docs_manual + docs_alarm

In [68]:
len(docs_list)

3315

In [78]:
from langchain.docstore.document import Document

documents_manual = [
    Document(page_content=text, metadata={"source": "TULUS manual software"})
    for text in docs_manual
]

documents_alarm = [
    Document(page_content=text, metadata={"source": "TULUS - Alarms"})
    for text in docs_alarm
]

docs = documents_manual + documents_alarm

In [94]:
output_pickle_file = "docs.pkl"
with open(output_pickle_file, 'wb') as f:
    # Dump the concatenated list into the pickle file
    pickle.dump(docs, f)

In [79]:
# Initialize the embeddings and vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store_chroma = Chroma.from_documents(docs,embeddings)

In [80]:
vector_store_faiss = FAISS.from_documents(docs,embeddings)



In [81]:
# Define a prompt template for the language model
prompt_template = PromptTemplate(input_variables=["context", "question"],
                                 template="Given the following context: {context}, answer the question: {question}")


In [85]:
retriever=vector_store_chroma.as_retriever(search_type="similarity",search_kwargs={"k":4})
# retriever=vector_store_chroma.as_retriever(search_type="similarity")

In [31]:

retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt_template}
)

In [87]:
# Function to retrieve answer and chunks used
def get_answer_and_chunks(query):
    # Use the qa_chain to get the answer and source documents
    
    result = retrievalQA.invoke({"query": query})
    
    # Extract the answer and source documents
    answer = result["result"]
    source_documents = result["source_documents"]
    
    return answer, source_documents

# Define a query and retrieve the answer along with chunks
query = "how many alari Ids are there in tulus"
answer, source_docs = get_answer_and_chunks(query)

print("Answer:", answer)
print("Chunks Used:")


if source_docs:
    for index,chunk in enumerate(source_docs):
        print("Document :", index)
        # print(chunk.page_content)
        print("Metadata:", chunk.metadata)
        print()
else:
    print("No chunks used")

Answer: There are 4 Alarm IDs mentioned in the provided context:

1. Alarm ID: 32/147
2. Alarm ID: 352
3. Alarm ID: 22/44
4. Alarm ID: 0/1282
Chunks Used:
Document : 0
Metadata: {'source': 'TULUS - Alarms'}

Document : 1
Metadata: {'source': 'TULUS - Alarms'}

Document : 2
Metadata: {'source': 'TULUS - Alarms'}

Document : 3
Metadata: {'source': 'TULUS - Alarms'}



# Evaluation

## replace index in original dataset

In [55]:
import pandas as pd
file_path='/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/main_manual_file_with_index (1).csv'
df=pd.read_csv(file_path,sep=';')

In [57]:
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/data/chunks.pkl"
with open(pickle_file, "rb") as file:
    chunks = pickle.load(file)

In [ ]:
def replace_context(df, new_context_list):
    # Replace each context value based on the context_index
    df['context'] = df['closest_text_index'].apply(lambda x: new_context_list[x])
    return df

# Apply the function
df = replace_context(df, chunks)


In [68]:
df_main= df.drop(columns=["correct_answers","incorrect_answers","closest_text_index","category"])

In [71]:
df_main.rename(columns={"best_answer":"answer"},inplace=True)
df_main.rename(columns={"context":"contexts"},inplace=True)


In [76]:
df_main

,contexts,question,answer
0,\n2. TULUS user interface\n2.1 Introduction\n...,What is the main purpose of the Tulus software?,The main purpose of the Tulus software is to m...
1,\n2. TULUS user interface\n2.1 Introduction\n...,What are some key components of the Tulus user...,Some key components of the Tulus user interfac...
2,\n2. TULUS user interface\n2.1 Introduction\n...,What functions are available on the vertical t...,The vertical toolbar in the Tulus interface in...
3,\n2. TULUS user interface\n2.1 Introduction\n...,What information is displayed in the notificat...,The notification area displays messages about ...
4,\n2. TULUS user interface\n2.1 Introduction\n...,What kind of controls and information can be f...,"The ""Customer"" section of the Tulus interface ..."


## RAGAS Evaluation

### Faithfulness 
This measures the factual consistency of the generated answer against the given context. It is calculated from answer and retrieved context. The answer is scaled to (0,1) range. Higher the better.
The generated answer is regarded as faithful if all the claims that are made in the answer can be inferred from the given context. To calculate this a set of claims from the generated answer is first identified. Then each one of these claims are cross checked with given context to determine if it can be inferred from given context or not. The faithfulness score is given by divided by (1) 

Faithfulness score = Number of claims in the generated answer that can be inferred from given context / Total number of claims in the generated answer|

### Answer Relevance

The evaluation metric, Answer Relevancy, focuses on assessing how pertinent the generated answer is to the given prompt. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy. This metric is computed using the question, the context and the answer.

### Context Precision

Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question, ground_truth and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.

### Context Relevancy

This metric gauges the relevancy of the retrieved context, calculated based on both the question and contexts. The values fall within the range of (0, 1), with higher values indicating better relevancy.
Ideally, the retrieved context should exclusively contain essential information to address the provided query. To compute this, we initially estimate the value of S| by identifying sentences within the retrieved context that are relevant for answering the given question. The final score is determined by the following formula:

context relevancy = |S| / |Total number of sentences in retrieved context|

### Context Recall 

Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.

To estimate context recall from the ground truth answer, each sentence in the ground truth answer is analyzed to determine whether it can be attributed to the retrieved context or not. In an ideal scenario, all sentences in the ground truth answer should be attributable to the retrieved context.

The formula for calculating context recall is as follows:


context recall = |GT sentences that can be attributed to context| / Number of sentences in GT

### Answer Correctness

The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness.

Answer correctness encompasses two critical aspects: semantic similarity between the generated answer and the ground truth, as well as factual similarity. These aspects are combined using a weighted scheme to formulate the answer correctness score. Users also have the option to employ a ‘threshold’ value to round the resulting score to binary, if desired.

In [88]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)


from ragas.metrics.critique import harmfulness
from ragas import evaluate

In [95]:
from tqdm import tqdm
import pandas as pd
from datasets import Dataset



rag_dataset = []

def create_ragas_dataset(df_main,column_name="answer"):
  for index, row in tqdm(df_main.iterrows(), total=df_main.shape[0]):
    answer, source_docs = get_answer_and_chunks(row["question"])
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer,
         "contexts" : [context.page_content for context in source_docs],
         "ground_truths" : [row[column_name]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset



In [96]:
def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

# Test Set Generation

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(docs, 6000, distributions) 
testset.to_pandas()

                                                                   Exception in thread Thread-90:
Traceback (most recent call last):
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1191, in __iter__
    self.update(n - last_print_n)
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1242, in update
    self.refresh(lock_args=self.lock_args)
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1495, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 459, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-pac

In [58]:
df3=pd.DataFrame(testset.to_pandas())
df3.to_csv("testset.csv",index=False)

In [53]:
basic_qa_ragas_dataset = create_ragas_dataset(pd.DataFrame(testset.to_pandas()),"ground_truth")

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [03:50<00:00, 23.00s/it]


In [54]:
basic_qa_ragas_dataset.to_pandas()

,question,answer,contexts,ground_truths
0,What are the different functional areas within...,Based on the provided screenshots and descript...,[On the topleft there is a vertical navigation...,[The different functional areas within the app...
1,What can be accessed through the separate menu...,Through the separate menu in the Tulus system ...,[6. TULUS System management and settings\nThe...,[The tools needed for general management of th...
2,What is the purpose of the dropdown menus in t...,The purpose of the dropdown menus in the inter...,[The interface is divided into several section...,[The purpose of the dropdown menus in the inte...
3,What is the current status of the FPServiceBro...,The current status of the FPServiceBroker serv...,[The image appears to be a screenshot of a gra...,[The current status of the FPServiceBroker ser...
4,What is the purpose of the hierarchical struct...,The purpose of the hierarchical structure in t...,[On the top left corner there is a menu bar wi...,[The purpose of the hierarchical structure in ...
5,What does the license management tool do in th...,The license management tool in the software in...,[An activated license is valid only for the so...,[The license management tool in the software i...
6,What is the purpose of the main menu in the so...,The main menu in the software for laser operat...,[laser. 2 Play/Stop Button Start/stop a task l...,[The main menu in the software for laser opera...
7,What does the Cancel button do in the producti...,The Cancel button in the production order mana...,[At the bottom there are two buttons Add highl...,[The Cancel button allows the user to exit wit...
8,What is the purpose of MDA mode in the control...,"According to the text, MDA mode allows to run ...",[The center of the interface displays operatio...,[MDA mode allows for running utility programs ...
9,How does the hierarchical structure in the nav...,The hierarchical structure in the navigation p...,[On the top left corner there is a menu bar wi...,[The hierarchical structure in the navigation ...


In [56]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 70/70 [01:56<00:00,  1.66s/it]


In [57]:
basic_qa_result

{'context_precision': 0.9917, 'faithfulness': 0.7871, 'answer_relevancy': 0.9716, 'context_recall': 1.0000, 'context_relevancy': 0.0255, 'answer_correctness': 0.6130, 'answer_similarity': 0.9552}

In [33]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 70/70 [02:33<00:00,  2.19s/it]


In [34]:
basic_qa_result

{'context_precision': 0.9833, 'faithfulness': 0.9583, 'answer_relevancy': 0.9720, 'context_recall': 0.9000, 'context_relevancy': 0.0325, 'answer_correctness': 0.6566, 'answer_similarity': 0.9558}

In [ ]:
import os
import random
import numpy as np
import logging
import json
import pickle
from dotenv import load_dotenv
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_groq import ChatGroq
from tqdm import tqdm
import pandas as pd
from datasets import Dataset
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity,
)
from langchain.retrievers.document_compressors import FlashrankRerank
from ragas.metrics.critique import harmfulness
from ragas import evaluate
from ragatouille import RAGPretrainedModel
from langchain.retrievers import ContextualCompressionRetriever

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ensure the correct forward reference
# FlashrankRerank.update_forward_refs(Ranker=RankerModel)

# Now, you can use the FlashrankRerank with the RankerModel
compressor = FlashrankRerank(model_name="ms-marco-MultiBERT-L-12")


In [48]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ValidationError: 1 validation error for FlashrankRerank
client
  instance of RankerModel expected (type=type_error.arbitrary_type; expected_arbitrary_type=RankerModel)

In [26]:
from flashrank import Ranker, RerankRequest

# Small (~34MB), slightly slower & best performance (ranking precision).
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")

In [60]:
from flashrank import Ranker, RerankRequest
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
from langchain_cohere import CohereRerank
import os

load_dotenv()

COHERE_API_KEY = os.getenv("COHERE_API_KEY")

embeddings = OpenAIEmbeddings()

vector_store = FAISS.from_documents(docs, embeddings)

retriever = vector_store.as_retriever(
            search_type="similarity", search_kwargs={"k": 10})





INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [87]:
import os
HUGGINGFACE_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")
groq_api_key=os.getenv("GRORQ_API_KEY")

In [88]:
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_community.llms import HuggingFaceHub
from langchain_groq import ChatGroq
import pickle

llm=ChatGroq(groq_api_key=groq_api_key, model_name='llama3-70b-8192')


In [92]:
result = llm.invoke("what is the capital of France")



INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_community.llms import HuggingFaceHub
from langchain_groq import ChatGroq
import pickle




In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")
groq_api_key = os.environ["GROQ_API_KEY"]

# Naive RAG

In [ ]:
hf=HuggingFaceHub(
    repo_id="meta-llama/Meta-Llama-3-70B",
    model_kwargs={"temperature":0.1}

)

/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama3-70b-8192")

In [ ]:
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/data/chunks.pkl"
with open(pickle_file, "rb") as file:
    docs_manual = pickle.load(file)

In [ ]:
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/data/alarm_info_list.pkl"
with open(pickle_file, "rb") as file:
    docs_alarm = pickle.load(file)

In [ ]:
docs_list = docs_manual + docs_alarm

In [ ]:
len(docs_list)

3315

In [ ]:
from langchain.docstore.document import Document

documents_manual = [
    Document(page_content=text, metadata={"source": "TULUS manual software"})
    for text in docs_manual
]

documents_alarm = [
    Document(page_content=text, metadata={"source": "TULUS - Alarms"})
    for text in docs_alarm
]

docs = documents_manual + documents_alarm

In [ ]:
output_pickle_file = "docs.pkl"
with open(output_pickle_file, 'wb') as f:
    # Dump the concatenated list into the pickle file
    pickle.dump(docs, f)

In [ ]:
# Initialize the embeddings and vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store_chroma = Chroma.from_documents(docs,embeddings)

In [ ]:
vector_store_faiss = FAISS.from_documents(docs,embeddings)



In [ ]:
# Define a prompt template for the language model
prompt_template = PromptTemplate(input_variables=["context", "question"],
                                 template="Given the following context: {context}, answer the question: {question}")


In [ ]:
retriever=vector_store_chroma.as_retriever(search_type="similarity",search_kwargs={"k":4})
# retriever=vector_store_chroma.as_retriever(search_type="similarity")

In [ ]:

retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt_template}
)

In [ ]:
# Function to retrieve answer and chunks used
def get_answer_and_chunks(query):
    # Use the qa_chain to get the answer and source documents
    
    result = retrievalQA.invoke({"query": query})
    
    # Extract the answer and source documents
    answer = result["result"]
    source_documents = result["source_documents"]
    
    return answer, source_documents

# Define a query and retrieve the answer along with chunks
query = "how many alari Ids are there in tulus"
answer, source_docs = get_answer_and_chunks(query)

print("Answer:", answer)
print("Chunks Used:")


if source_docs:
    for index,chunk in enumerate(source_docs):
        print("Document :", index)
        # print(chunk.page_content)
        print("Metadata:", chunk.metadata)
        print()
else:
    print("No chunks used")

Answer: There are 4 Alarm IDs mentioned in the provided context:

1. Alarm ID: 32/147
2. Alarm ID: 352
3. Alarm ID: 22/44
4. Alarm ID: 0/1282
Chunks Used:
Document : 0
Metadata: {'source': 'TULUS - Alarms'}

Document : 1
Metadata: {'source': 'TULUS - Alarms'}

Document : 2
Metadata: {'source': 'TULUS - Alarms'}

Document : 3
Metadata: {'source': 'TULUS - Alarms'}



# Evaluation

## replace index in original dataset

In [ ]:
import pandas as pd
file_path='/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/main_manual_file_with_index (1).csv'
df=pd.read_csv(file_path,sep=';')

In [ ]:
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/data/chunks.pkl"
with open(pickle_file, "rb") as file:
    chunks = pickle.load(file)

In [ ]:
def replace_context(df, new_context_list):
    # Replace each context value based on the context_index
    df['context'] = df['closest_text_index'].apply(lambda x: new_context_list[x])
    return df

# Apply the function
df = replace_context(df, chunks)


In [ ]:
df_main= df.drop(columns=["correct_answers","incorrect_answers","closest_text_index","category"])

In [ ]:
df_main.rename(columns={"best_answer":"answer"},inplace=True)
df_main.rename(columns={"context":"contexts"},inplace=True)


In [ ]:
df_main

,contexts,question,answer
0,\n2. TULUS user interface\n2.1 Introduction\n...,What is the main purpose of the Tulus software?,The main purpose of the Tulus software is to m...
1,\n2. TULUS user interface\n2.1 Introduction\n...,What are some key components of the Tulus user...,Some key components of the Tulus user interfac...
2,\n2. TULUS user interface\n2.1 Introduction\n...,What functions are available on the vertical t...,The vertical toolbar in the Tulus interface in...
3,\n2. TULUS user interface\n2.1 Introduction\n...,What information is displayed in the notificat...,The notification area displays messages about ...
4,\n2. TULUS user interface\n2.1 Introduction\n...,What kind of controls and information can be f...,"The ""Customer"" section of the Tulus interface ..."


## RAGAS Evaluation

### Faithfulness 
This measures the factual consistency of the generated answer against the given context. It is calculated from answer and retrieved context. The answer is scaled to (0,1) range. Higher the better.
The generated answer is regarded as faithful if all the claims that are made in the answer can be inferred from the given context. To calculate this a set of claims from the generated answer is first identified. Then each one of these claims are cross checked with given context to determine if it can be inferred from given context or not. The faithfulness score is given by divided by (1) 

Faithfulness score = Number of claims in the generated answer that can be inferred from given context / Total number of claims in the generated answer|

### Answer Relevance

The evaluation metric, Answer Relevancy, focuses on assessing how pertinent the generated answer is to the given prompt. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy. This metric is computed using the question, the context and the answer.

### Context Precision

Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question, ground_truth and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.

### Context Relevancy

This metric gauges the relevancy of the retrieved context, calculated based on both the question and contexts. The values fall within the range of (0, 1), with higher values indicating better relevancy.
Ideally, the retrieved context should exclusively contain essential information to address the provided query. To compute this, we initially estimate the value of S| by identifying sentences within the retrieved context that are relevant for answering the given question. The final score is determined by the following formula:

context relevancy = |S| / |Total number of sentences in retrieved context|

### Context Recall 

Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.

To estimate context recall from the ground truth answer, each sentence in the ground truth answer is analyzed to determine whether it can be attributed to the retrieved context or not. In an ideal scenario, all sentences in the ground truth answer should be attributable to the retrieved context.

The formula for calculating context recall is as follows:


context recall = |GT sentences that can be attributed to context| / Number of sentences in GT

### Answer Correctness

The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness.

Answer correctness encompasses two critical aspects: semantic similarity between the generated answer and the ground truth, as well as factual similarity. These aspects are combined using a weighted scheme to formulate the answer correctness score. Users also have the option to employ a ‘threshold’ value to round the resulting score to binary, if desired.

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)


from ragas.metrics.critique import harmfulness
from ragas import evaluate

In [ ]:
from tqdm import tqdm
import pandas as pd
from datasets import Dataset



rag_dataset = []

def create_ragas_dataset(df_main,column_name="answer"):
  for index, row in tqdm(df_main.iterrows(), total=df_main.shape[0]):
    answer, source_docs = get_answer_and_chunks(row["question"])
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer,
         "contexts" : [context.page_content for context in source_docs],
         "ground_truths" : [row[column_name]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset



In [ ]:
def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

# Test Set Generation

In [1]:
import pickle
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/docs.pkl"
with open(pickle_file, "rb") as file:
    docs = pickle.load(file)

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(docs, 6000, distributions) 
testset.to_pandas()

                                                                   Exception in thread Thread-90:
Traceback (most recent call last):
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1191, in __iter__
    self.update(n - last_print_n)
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1242, in update
    self.refresh(lock_args=self.lock_args)
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 1495, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/std.py", line 459, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-pac

In [ ]:
df3=pd.DataFrame(testset.to_pandas())
df3.to_csv("testset.csv",index=False)

In [24]:
import pandas as pd

# Reading the CSV files
df1 = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/testset_alarms.csv')
df2 = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/testset_manual.csv')
df3 = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/trainset_manual.csv')

# Combining the dataframes
combined_df1 = pd.concat([df1, df2, df3], ignore_index=True)
combined_df1.drop(['evolution_type', 'metadata','episode_done'], axis=1, inplace=True)


In [32]:
combined_df1

,question,contexts,ground_truth
0,What could be the possible issues when a timeo...,"['Alarm ID is: 3402, Source type is: PLC alarm...",When a timeout occurs on the 'jaws open comman...
1,What is the issue reported in the alarm relate...,"['Alarm ID is: 23/136, Source type is: CNC ala...",The issue reported in the alarm related to the...
2,What does 'Undefined DPT for probing cycle' re...,"['Alarm ID is: 23/251, Source type is: CNC ala...","In the context of a CNC alarm, 'Undefined DPT ..."
3,What is the recommended action to resolve the ...,"['Alarm ID is: 22/303, Source type is: CNC ala...",NaN
4,What type of information is typically indicate...,"['Alarm ID is: 21/43, Source type is: CNC alar...",CNC alarms typically indicate technical inform...
...,...,...,...
214,What options are available under 'Pallet' in L...,['9. TULUS Laser machine\nWith the Laser Mach...,NaN
215,"What app manages Tulus license activation, usi...",['The arrows between the screenshots suggest t...,The License Management application is used to ...
216,What's TIP CHANGER's role in calibrating the s...,['The interface is divided into several sectio...,NaN
217,TULUS interface: Which feature lets user manua...,['9. TULUS Laser machine\nWith the Laser Mach...,NaN


In [7]:
# Reading the CSV file
df_alarms = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/qa_pairs_alarm.csv')
df2_manual = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/qa_pairs_manual.csv')


df_alarms= df_alarms[:650]



116

In [35]:

# Deleting the columns (replace 'column_to_delete1' and 'column_to_delete2' with actual column names)
df_alarms.drop(['category', 'correct_answers','incorrect_answers'], axis=1, inplace=True)
df2_manual.drop(['category', 'correct_answers','incorrect_answers'], axis=1, inplace=True)

# Renaming the columns (replace 'old_column_name1' and 'old_column_name2' with actual column names)
# and provide the new names 'new_column_name1' and 'new_column_name2'
df_alarms.rename(columns={'best_answer': 'ground_truth','context': 'contexts'}, inplace=True)
df2_manual.rename(columns={'best_answer': 'ground_truth','context': 'contexts'}, inplace=True)


combined_df = pd.concat([df_alarms, df2_manual], ignore_index=True)



In [36]:
combined_df_main = pd.concat([combined_df, combined_df1], ignore_index=True)


In [37]:
# write the code to drop null values from the dataframe and reset the index
combined_df_main.dropna(inplace=True)
# Resetting the index
combined_df_main.reset_index(drop=True, inplace=True)

In [38]:
combined_df_main

,contexts,question,ground_truth
0,The text content describes an alarm triggered ...,What should be done if the axes brake is manua...,You should bring the brakes release key on the...
1,The text content describes an alarm triggered ...,Why does the alarm trigger when the axes brake...,The alarm triggers because the cycle is runnin...
2,The alarm indicates that the line air pressure...,What should you do if the line air pressure is...,You should use the pressure regulator in the p...
3,The alarm indicates that the line air pressure...,What is the nominal inlet value for the line a...,The nominal inlet value for the line air press...
4,Alarm ID: 3 indicates that the Over-travel BY-...,Why is it important to activate the Over-trave...,It is important to activate the Over-travel BY...
...,...,...,...
1454,['Yellow trial license. The buttons on the Lic...,"What does Earth-icon on Licenses page signify,...",The Earth-icon on the Licenses page is a symbo...
1455,['6. TULUS System management and settings\nTh...,"Where in Tulus interface, apart from help view...",The tools needed for general management of the...
1456,['Each box has an arrow pointing downward. Bel...,How do you set up and oversee various backups ...,To set up and oversee various backups in a dat...
1457,['The arrows between the screenshots suggest t...,"What app manages Tulus license activation, usi...",The License Management application is used to ...


In [39]:
# Saving the modified dataframe to a new CSV file
combined_df_main.to_csv('full_combined_dataset_main.csv', index=False)

In [23]:
import pandas as pd
combined_df_main = pd.read_csv("/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/full_combined_dataset_main.csv")

In [28]:
# Function to sample 100 records from specified ranges
def sample_records(dataframe, ranges, sample_size):
    sampled_data = pd.DataFrame()
    for start, end in ranges:
        sampled_data = pd.concat([sampled_data, dataframe.iloc[start:end].sample(n=sample_size, random_state=1)])
    return sampled_data

# Define the ranges
ranges = [(0, 650), (650, 1275), (1275, 1458)]

# Get the sampled records
sampled_df = sample_records(combined_df_main, ranges, 100)

In [29]:
sampled_df.reset_index(drop=True, inplace=True)
sampled_df

,contexts,question,ground_truth
0,"When the Start-up mode is active, the automati...",How can you disable the Start-up mode on the m...,You can disable the Start-up mode from the Mac...
1,The text provides a description and remedy for...,What steps should you take if a laser module i...,"Turn off the laser power supply unit, turn the..."
2,Alarm ID 2321 indicates that the HEAD Precitec...,What should you do if the HEAD Precitec does n...,Check the stability of the metal down head dur...
3,An alarm (ID: 2309) for the HEAD Precitec indi...,What does Alarm ID 2309 indicate in the HEAD P...,It indicates an internal system error in the d...
4,The text describes an alarm with ID 2193 indic...,What is missing from the alarm description for...,The alarm description for Alarm ID 2193 is mis...
...,...,...,...
295,['Refresh It is used to update the license pag...,What is the sequence of steps for activating a...,The sequence of steps for activating a new lic...
296,['The GUI is colorful and divided into several...,How do you access nozzle mgmt/switch control f...,"In the given context, the NozzleLibraryHandler..."
297,['Commands with an icon that appears to be a t...,"What actions allow customization of entries, a...","In the Tulus help viewer app, customization of..."
298,['8. TULUS Control panel\nCertain basic funct...,What is the purpose of the Control Panel in th...,The Control Panel in the Tulus user interface ...


In [31]:
sampled_df.to_csv('full_dataset_sampled.csv', index=False)


In [2]:
df.columns

Index(['contexts', 'question', 'ground_truth'], dtype='object')

In [84]:
import pandas as pd
# Reading the CSV file
df= pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/full_combined_dataset_main.csv')



In [85]:
import pandas as pd

# Function to select random rows from given index range
def select_random_from_range(df, start_idx, end_idx, n=300):
    # Ensure the selection range does not exceed DataFrame bounds
    selection_range = df.iloc[start_idx:end_idx]
    # Randomly select 'n' rows from this range
    random_selection = selection_range.sample(n=n)
    return random_selection

# Selecting 300 random rows from the first range
random_selection_1 = select_random_from_range(df, start_idx=0, end_idx=649, n=456)

# Selecting 300 random rows from the second range
random_selection_2 = select_random_from_range(df, start_idx=650, end_idx=1273, n=359)

random_selection_3 = df[1274:]


# Concatenating the two selections
final_selection = pd.concat([random_selection_1, random_selection_2, random_selection_3], ignore_index=True)

# Display the final concatenated DataFrame


In [86]:
final_selection

,contexts,question,ground_truth
0,The alarm with ID 933 indicates a timeout sign...,What does Alarm ID: 933 indicate?,Alarm ID: 933 indicates a timeout signal PS-Ac...
1,An alarm with ID 351 indicates that the Drag 1...,What should you do if the Drag 1 motor enable ...,You should check the electrical motor contacto...
2,"The content describes an alarm with ID 2187, i...",What does Alarm ID 2187 signify in the context...,Alarm ID 2187 signifies a timeout issue with t...
3,The text describes an alarm (ID: 1105) that in...,What should a user do when they encounter the ...,Contact technical support.
4,The text content describes an alarm with ID 35...,What should be checked if the feedback for ena...,The electrical motor contactor circuit should ...
...,...,...,...
995,['Yellow trial license. The buttons on the Lic...,"What does Earth-icon on Licenses page signify,...",The Earth-icon on the Licenses page is a symbo...
996,['6. TULUS System management and settings\nTh...,"Where in Tulus interface, apart from help view...",The tools needed for general management of the...
997,['Each box has an arrow pointing downward. Bel...,How do you set up and oversee various backups ...,To set up and oversee various backups in a dat...
998,['The arrows between the screenshots suggest t...,"What app manages Tulus license activation, usi...",The License Management application is used to ...


In [87]:
final_selection.to_csv('final_selection.csv', index=False)

In [42]:
sampled_df[:]

,contexts,question,ground_truth,question,contexts,ground_truth
0,"When the Start-up mode is active, the automati...",How can you disable the Start-up mode on the m...,You can disable the Start-up mode from the Mac...,NaN,NaN,NaN
1,The text provides a description and remedy for...,What steps should you take if a laser module i...,"Turn off the laser power supply unit, turn the...",NaN,NaN,NaN
2,Alarm ID 2321 indicates that the HEAD Precitec...,What should you do if the HEAD Precitec does n...,Check the stability of the metal down head dur...,NaN,NaN,NaN
3,An alarm (ID: 2309) for the HEAD Precitec indi...,What does Alarm ID 2309 indicate in the HEAD P...,It indicates an internal system error in the d...,NaN,NaN,NaN
4,The text describes an alarm with ID 2193 indic...,What is missing from the alarm description for...,The alarm description for Alarm ID 2193 is mis...,NaN,NaN,NaN
...,...,...,...,...,...,...
596,The text describes a software interface used i...,"What does the ""Sheets in total"" field indicate?","The ""Sheets in total"" field indicates the tota...",NaN,NaN,NaN
597,The interface displays a user interface for ma...,What happens if the pallets in the Loading and...,If the pallets in the Loading and Working area...,NaN,NaN,NaN
598,The text outlines the process of creating an M...,What action should be performed after clicking...,Select the command to be added and set its par...,NaN,NaN,NaN
599,The text describes a software program’s user i...,What are the columns labeled in the table belo...,"The columns are labeled Time, Type, Process ow...",NaN,NaN,NaN


Dataset


First 380 is alarms
Second 284 is manual
Third 44 is alarms
The rest is manual: 140


Manual: 424
Alarms: 424

In [5]:
from datasets import Dataset 
from ragas.metrics import context_recall
from ragas import evaluate
import toml
import os

with open("/Users/hadiibrahim/Dev/prima-power-hmi-assistant/config.toml", "r") as f:
    config = toml.load(f)

# Access values from the config

os.environ["OPENAI_API_KEY"] = config["env"]["OPENAI_API_KEY"]

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

In [8]:
import asyncio
from ragas import evaluate
from ragas.metrics import context_recall
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq import ChatGroq


embeddings = HuggingFaceBgeEmbeddings(
                model_name="BAAI/bge-small-en-v1.5",  # or sentence-trainsformers/all-MiniLM-L6-v2
                model_kwargs={"device": "cpu"},
                encode_kwargs={"normalize_embeddings": True},
            )
llm = ChatGroq(
                groq_api_key='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',
                model='llama-3.1-70b-Versatile',
                temperature='0',
            )
for attempt in range(3):
    try:
        score = evaluate(dataset, metrics=[context_recall],llm=llm,embeddings=embeddings)
        df_score = score.to_pandas()
        print(df_score)
        break  # Exit loop if successful
    except asyncio.TimeoutError:
        print(f"Attempt {attempt + 1} timed out. Retrying...")
    except Exception as e:
        print(f"An error occurred: {e}")
        break  # Exit loop on other exceptions

Task exception was never retrieved
future: <Task finished name='Task-3' coro=<as_completed.<locals>.sema_coro() done, defined at /Users/hadiibrahim/anaconda3/envs/prima-env/lib/python3.12/site-packages/ragas/executor.py:32> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/hadiibrahim/anaconda3/envs/prima-env/lib/python3.12/asyncio/tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "/Users/hadiibrahim/anaconda3/envs/prima-env/lib/python3.12/site-packages/ragas/metrics/_context_recall.py", line 168, in _ascore
    results = await self.llm.generate(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadiibrahim/anaconda3/envs/prima-env/lib/python3.12/site-packages/ragas/llms/base.py", line 95, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadiibrahim/anaconda3/envs/prima-env/lib/python3.12/site-packages/tenacity/asyncio/__init__.py", line 189, in async_wrapped


                         question  \
0  When was the first super bowl?   
1   Who won the most super bowls?   

                                              answer  \
0       The first superbowl was held on Jan 15, 1967   
1  The most super bowls have been won by The New ...   

                                            contexts  \
0  [The First AFL–NFL World Championship Game was...   
1  [The Green Bay Packers...Green Bay, Wisconsin....   

                                        ground_truth  context_recall  
0   The first superbowl was held on January 15, 1967             1.0  
1  The New England Patriots have won the Super Bo...             0.0  


In [9]:
df_score

,question,answer,contexts,ground_truth,context_recall
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967",[The First AFL–NFL World Championship Game was...,"The first superbowl was held on January 15, 1967",1.0
1,Who won the most super bowls?,The most super bowls have been won by The New ...,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...,0.0


In [10]:
import pandas as pd

# Reading the CSV file
df= pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/ragas_dataset.csv')

In [21]:
# turn a string to an array
df['contexts'] = df['contexts'].apply(lambda x: x[1:-1].split(', '))


In [30]:
len(df['contexts'][7])

4

In [80]:
import pandas as pd
import json

# Step 1: Load your DataFrame
# Assuming your DataFrame is stored in a CSV file named 'data.csv'
df = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/data/final_selection.csv')

# Step 2: Load and parse the JSON file
# Assuming your JSON file is named 'data.json'
with open('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/models_saved/2024-09-01T11:49+02:00-rag_exp/results.json', 'r') as file:
    json_data = json.load(file)

# Step 3: Extract the first element of each array within the 'answers' array
# Assuming 'answers' is the key in the JSON data
first_elements = [item[0] for item in json_data['answers']]
df['contexts'] = df['contexts'].apply(lambda x: [x])

# Step 4: Add the extracted elements as a new column in the DataFrame
df['answer'] = first_elements

# Step 5: Optionally, save the updated DataFrame back to a CSV file
df.to_csv('updated_data1.csv', index=False)

# Display the updated DataFrame


In [82]:
from datasets import Dataset
df_dict = df.to_dict(orient='list')
rag_eval_dataset = Dataset.from_dict(df_dict)

In [75]:
rag_eval_dataset['contexts'][900]

"['3. TULUS  Mda\\nThe MDA Manual Data Automatic function allows to create modify and run MDA programs\\nsingle commands or sets of commands. Given the complexity of the part program structure\\nmovement programs are generally managed but not the cutting programs. To access the MDA page click the MDA button in the main menu. The Play button on\\nthe interface will be modified to highlight operation of an MDA program. The buttons at the top of the MDA page are used to perform all the operations necessary for\\ncreating/manipulating a MDA program. The image shows a screenshot of a software application with a graphical user interface GUI.']"

In [86]:
Dataset.from_dict(rag_eval_dataset[0:1])

Dataset({
    features: ['contexts', 'question', 'ground_truth', 'answer'],
    num_rows: 1
})

In [58]:

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(
                model_name="BAAI/bge-small-en-v1.5",  # or sentence-trainsformers/all-MiniLM-L6-v2
                model_kwargs={"device": "cpu"},
                encode_kwargs={"normalize_embeddings": True},
            )

In [103]:


from langchain_groq import ChatGroq
llm = ChatGroq(
                groq_api_key="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",
                model="llama-3.1-70b-Versatile",
                temperature=0,
            )

In [63]:
df.columns

Index(['contexts', 'question', 'ground_truth', 'answers'], dtype='object')

In [104]:
import time
import traceback
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

# Function to save progress
def save_progress(result, save_path="evaluation_progress.csv"):
    df = pd.DataFrame(result)
    df.to_csv(save_path, index=False)

# Function to load progress
def load_progress(save_path="evaluation_progress.csv"):
    try:
        df = pd.read_csv(save_path)
        return df.to_dict(orient='list')
    except FileNotFoundError:
        return {metric: [] for metric in ["answer_relevancy", "faithfulness", "context_recall", "context_precision", "answer_correctness","answer_similarity"]}

# Define a function for evaluation with retry mechanism and progress saving
def evaluate_with_resume(dataset, metrics, llm, embeddings, retries=5, sleep_time=300, save_path="evaluation_progress.csv"):
    progress = load_progress(save_path)
    start_index = len(progress)  # Start from where we left off

    while True:
        try:
            for i in range(start_index, len(dataset)):
                # Evaluate a single item
                result = evaluate(
                    Dataset.from_dict(dataset[i:i+1]),  # Evaluate one item at a time
                    metrics=metrics,
                    llm=llm,
                    embeddings=embeddings,
                )

                # Append each metric result to the corresponding list in the progress dictionary
                for metric, value in result.items():
                    progress[metric].append(value)
                save_progress(progress, save_path)  # Save progress after each step

            return progress  # Return the full results if successful

        except Exception as e:
            print(f"Attempt {attempt + 1} failed at index {i} with error: {str(e)}")
            traceback.print_exc()

            if attempt + 1 == retries:
                raise

            print(f"Sleeping for {sleep_time} seconds before retrying...")
            time.sleep(sleep_time)
            print("Retrying...")

# List of metrics to be used for evaluation
metrics = [
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_correctness,
    answer_similarity,
]

# Run the evaluation with the resume and retry mechanism
try:
    result = evaluate_with_resume(
        rag_eval_dataset,
        metrics=metrics,
        llm=llm,
        embeddings=embeddings,
        save_path="evaluation_progress.csv"
    )
    print("Evaluation completed successfully.")
except Exception as e:
    print("Evaluation failed after multiple attempts.")
    print(str(e))


KeyboardInterrupt: 

In [1]:
import pickle
pickle_file= '/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/docs.pkl'
with open(pickle_file, "rb") as file:
            docs = pickle.load(file)

In [15]:
docs = [doc.page_content for doc in docs]

In [3]:
# Load the CSV file
import pandas as pd
file_path = '/Users/hadiibrahim/Dev/prima-power-hmi-assistant/ragas_dataset_rag_exp_naive.csv'  # Replace with your file path
df = pd.read_csv(file_path)

In [6]:
import pandas as pd
import ast

# Function to convert string to array of strings
def convert_to_array_of_strings(context_str):
    # Check if the string is formatted like an array (starts with [ and ends with ])
    if context_str.startswith('[') and context_str.endswith(']'):
        try:
            # Safely evaluate the string to a Python list
            context_list = ast.literal_eval(context_str)
            
            # Ensure all elements are strings
            if isinstance(context_list, list):
                context_list = [str(elem) for elem in context_list]
            else:
                # If it's not a list, treat the entire string as a single element
                context_list = [context_str]
        except (ValueError, SyntaxError):
            # If there's an error parsing, wrap the string as a single-element list
            context_list = [context_str]
    else:
        # If it's not an array-like string, treat it as a single-element list
        context_list = [context_str]
    
    return context_list

# Apply the conversion to the 'Real Context' column
df['contexts'] = df['contexts'].apply(convert_to_array_of_strings)

In [11]:
import pandas as pd
import ast

# Function to convert string to array of strings
def convert_to_array_of_strings(context_str):
    # Check if the string is formatted like an array (starts with [ and ends with ])
    if context_str.startswith('[') and context_str.endswith(']'):
        try:
            # Safely evaluate the string to a Python list
            context_list = ast.literal_eval(context_str)
            
            # Ensure all elements are strings
            if isinstance(context_list, list):
                context_list = [str(elem) for elem in context_list]
            else:
                # If it's not a list, treat the entire string as a single element
                context_list = [context_str]
        except (ValueError, SyntaxError):
            # If there's an error parsing, wrap the string as a single-element list
            context_list = [context_str]
    else:
        # If it's not an array-like string, treat it as a single-element list
        context_list = [context_str]
    
    return context_list

# Apply the conversion to the 'Real Context' column
df['original_contexts'] = df['original_contexts'].apply(convert_to_array_of_strings)

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize the model (you can choose any other pre-trained model)
model = SentenceTransformer('all-MiniLM-L6-v2')



# Compute embeddings for docs
doc_embeddings = model.encode(docs)



In [18]:
# Function to find the most similar document for each context array
def match_contexts(context_list):
    # Compute embeddings for the current list of contexts
    context_embeddings = model.encode(context_list)
    
    # Compute cosine similarities between each context and all documents
    similarities = cosine_similarity(context_embeddings, doc_embeddings)
    
    # Find the index of the most similar doc for each context
    matched_docs_indices = np.argmax(similarities, axis=1)
    
    # Get the matched docs based on the indices
    matched_docs = [docs[idx] for idx in matched_docs_indices]
    
    return matched_docs

# Apply the function to the 'contexts' column and create a new column 'matched_contexts'
df['matched_contexts'] = df['original_contexts'].apply(match_contexts)



In [2]:
import pandas as pd
df = pd.read_csv('/Users/hadiibrahim/Dev/prima-power-hmi-assistant/ragas_dataset_rag_exp_llm_chain.csv')

In [3]:
df

,question,answer,contexts,ground_truth,original_contexts
0,What does Alarm ID: 933 indicate?,There is no alarm information available for Al...,"['Alarm ID is: 23/363, Source type is: CNC ala...",Alarm ID: 933 indicates a timeout signal PS-Ac...,The alarm with ID 933 indicates a timeout sign...
1,What should you do if the Drag 1 motor enable ...,"According to the context, if the Drag 1 motor ...","[""Alarm ID is: 354, Source type is: PLC alarm/...",You should check the electrical motor contacto...,An alarm with ID 351 indicates that the Drag 1...
2,What does Alarm ID 2187 signify in the context...,"For Alarm ID 2187, the alarm text is ""Timeout ...","['Alarm ID is: 2164, Source type is: PLC alarm...",Alarm ID 2187 signifies a timeout issue with t...,"The content describes an alarm with ID 2187, i..."
3,What should a user do when they encounter the ...,"For Alarm ID 1105, the alarm information is as...","['Alarm ID is: 1105, Source type is: PLC alarm...",Contact technical support.,The text describes an alarm (ID: 1105) that in...
4,What should be checked if the feedback for ena...,"According to the context, if the feedback for ...","[""Alarm ID is: 354, Source type is: PLC alarm/...",The electrical motor contactor circuit should ...,The text content describes an alarm with ID 35...
...,...,...,...,...,...
636,What is the purpose of the fields labeled Text...,"The purpose of the fields labeled ""Text Descri...","['Alarm ID is: 42, Source type is: Storage for...",The fields labeled Text Description and Remedy...,The application features a toolbar with tabs l...
637,Which button is highlighted in the grid of but...,The Q button is highlighted in the grid of but...,['The background of the button is light grey a...,"The button for ST 2.25 is highlighted, indicat...",The interface is divided into several sections...
638,Where are the numbered boxes located?,"The numbered boxes are located at the top, arr...",['At the top there are three numbered boxes ar...,The numbered boxes are located at the top.,At the top there are three numbered boxes arra...
639,What kind of information can be found in the m...,The manuals for machines and devices likely co...,['Information on PLC and CNC versions. Informa...,The manuals provide instructions and informati...,The text content lists items such as showing t...
